In [3]:
import os
import time
import random
import numpy as np
import pickle as pk
import pandas as pd
from tqdm import tqdm
from operator import itemgetter
from collections import defaultdict
import torch
import jieba


# 准备好模型的参数
parameter = {
    'epoch':100,
    'batch_size':300,
    'embedding_dim':300,
    'hidden_size':128,
    'num_layers':2, 
    'dropout':0.1,
    #'cuda':torch.device('cuda'),#我电脑不支持
    'cuda':torch.device('cpu'),
    'lr':0.001,
    'max_len':50,
}

def build_dataSet(parameter):
    data_src = pd.read_csv('../../dataSet/data_src.csv')
    data_src = data_src[data_src['关系'] == 'question2answer']
    q,a = list(data_src.实体1),list(data_src.实体2)
    word2id = defaultdict(int)
    word2id['<PAD>'] = 0
    word2id['<UNK>'] = 0
    qa_list = {}
    for ind in range(len(q)):
        q_cut = list(q[ind])
        a_cut = list(a[ind])
        if q[ind] not in qa_list:
            qa_list[q[ind]] = [q_cut,a_cut]
        else:
            qa_list[q[ind]] += [a_cut]
        for i in q_cut:
            word2id[i] += 1
        for i in a_cut:
            word2id[i] += 1
    qa_list = list(qa_list.values())
    parameter['qa_list'] = qa_list
    parameter['word2id'] = dict(zip(word2id.keys(),range(len(word2id))))
    parameter['id2word'] = dict(zip(range(len(word2id)),word2id.keys()))
    parameter['word_size'] = len(word2id)
    
def sample(n,parameter,neg_sample_num):
    neg_sample = []
    q_size = len(parameter['qa_list'])
    while 1:
        sample_id = random.randint(0,q_size-1)
        if sample_id == n:
            continue
        neg_sample_answer = parameter['qa_list'][sample_id]
        a_id = random.randint(1,len(neg_sample_answer)-1)
        neg_sample.append(neg_sample_answer[a_id])
        if len(neg_sample) >= neg_sample_num:
            return neg_sample
        
def list2torch(a):
    return torch.from_numpy(np.array(a)).long().to(parameter['cuda'])
    
def batch_yield(parameter,shuffle = True):
    for train_epoch in range(parameter['epoch']):
        qa_list = parameter['qa_list']
        data = []
        for ind,i in enumerate(qa_list):
            q = i[0]
            p_a = i[1:]
            n_a = sample(ind,parameter,len(p_a))
            q = [q] * len(p_a)
            data += list(zip(q,p_a,n_a))
        if shuffle:
            random.shuffle(data)
        batch_q,batch_a,batch_n = [],[],[]
        seq_len_q,seq_len_a,seq_len_n = 0,0,0
        for (q,a,n) in tqdm(data):
            q = itemgetter(*q)(parameter['word2id'])
            a = itemgetter(*a)(parameter['word2id'])
            n = itemgetter(*n)(parameter['word2id'])
            q = list(q) if type(q) == type(()) else [q,0]
            a = list(a) if type(a) == type(()) else [a,0]
            n = list(n) if type(n) == type(()) else [n,0]
            q = q[:parameter['max_len']]
            a = a[:parameter['max_len']]
            n = n[:parameter['max_len']]
            if len(q) > seq_len_q:
                seq_len_q = len(q)
            if len(a) > seq_len_a:
                seq_len_a = len(a)
            if len(n) > seq_len_n:
                seq_len_n = len(n)
            batch_q.append(q)
            batch_a.append(a)
            batch_n.append(n)
            if len(batch_q) >= parameter['batch_size']:
                batch_q = [i+[0]*(seq_len_q-len(i)) for i in batch_q]
                batch_a = [i+[0]*(seq_len_a-len(i)) for i in batch_a]
                batch_n = [i+[0]*(seq_len_n-len(i)) for i in batch_n]
                yield list2torch(batch_q),list2torch(batch_a),list2torch(batch_n),None,False
                batch_q,batch_a,batch_n = [],[],[]
                seq_len_q,seq_len_a,seq_len_n = 0,0,0
        batch_q = [i+[0]*(seq_len_q-len(i)) for i in batch_q]
        batch_a = [i+[0]*(seq_len_a-len(i)) for i in batch_a]
        batch_n = [i+[0]*(seq_len_n-len(i)) for i in batch_n]
        yield list2torch(batch_q),list2torch(batch_a),list2torch(batch_n),train_epoch,False
        batch_q,batch_a,batch_n = [],[],[]
        seq_len_q,seq_len_a,seq_len_n = 0,0,0
    yield None,None,None,None,True
            
build_dataSet(parameter)
pk.dump(parameter,open('parameter.pkl','wb'))

In [4]:
train_yield = batch_yield(parameter)
test_q,test_a,test_n,_,_ = next(train_yield)
test_q,test_a,test_n

  0%|                                                                                                                              | 0/1390 [00:00<?, ?it/s]

(tensor([[ 438,  129,  197,  ...,    0,    0,    0],
         [ 282, 1054,  152,  ...,    0,    0,    0],
         [  15,   16,  200,  ...,    0,    0,    0],
         ...,
         [  60,   61,    0,  ...,    0,    0,    0],
         [ 907,  594,    7,  ...,    0,    0,    0],
         [  70,  292,  146,  ...,    0,    0,    0]]),
 tensor([[438, 343,  64,  ...,   0,   0,   0],
         [167, 580, 365,  ...,   0,   0,   0],
         [214,  79,  44,  ...,   0,   0,   0],
         ...,
         [ 60,  61, 288,  ..., 395, 361, 362],
         [539,  47, 266,  ...,   0,   0,   0],
         [868, 418, 341,  ...,   0,   0,   0]]),
 tensor([[121, 169, 385,  ..., 419,  29, 242],
         [209, 210, 211,  ...,  41,  43,  90],
         [319, 171, 387,  ...,   0,   0,   0],
         ...,
         [548, 159, 311,  ...,   0,   0,   0],
         [288, 104, 558,  ..., 141,  48, 553],
         [ 26,  13,  14,  ...,   0,   0,   0]]))

In [5]:
import torch.nn.functional as F # pytorch 激活函数的类
from torch import nn,optim # 构建模型和优化器

# 构建分类模型
class TextRNN(nn.Module):
    def __init__(self, parameter):
        super(TextRNN, self).__init__()
        embedding_dim = parameter['embedding_dim']
        hidden_size = parameter['hidden_size']
        num_layers = parameter['num_layers']
        dropout = parameter['dropout']
        word_size = parameter['word_size']
        self.embedding = nn.Embedding(word_size, embedding_dim, padding_idx=0)
        
        self.lstm_q = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)

        self.lstm_a = nn.LSTM(embedding_dim, hidden_size, num_layers, bidirectional=True, batch_first=True, dropout=dropout)


        
    def forward(self, q, a1,a2 = None):
        q_emd = self.embedding(q)
        q_emd,(h, c)= self.lstm_q(q_emd)
        q_emd = torch.max(q_emd,1)[0]

        a1_emd = self.embedding(a1)
        a1_emd,(h, c)= self.lstm_a(a1_emd)
        a1_emd = torch.max(a1_emd,1)[0]
        if a2 is not None:
            a2_emd = self.embedding(a2)
            a2_emd,(h, c)= self.lstm_a(a2_emd)
            a2_emd = torch.max(a2_emd,1)[0]
            return q_emd,a1_emd,a2_emd
        return F.cosine_similarity(q_emd,a1_emd,1,1e-8)

In [7]:
#test_model = TextRNN(parameter).cuda()#我电脑不支持
test_model = TextRNN(parameter)
test_model(test_q,test_a)

tensor([0.5519, 0.6497, 0.4228, 0.7707, 0.6180, 0.7856, 0.7394, 0.7570, 0.7016,
        0.7094, 0.5542, 0.7083, 0.6563, 0.6838, 0.5852, 0.5750, 0.6565, 0.6351,
        0.5930, 0.5578, 0.6730, 0.7508, 0.6937, 0.4789, 0.6029, 0.7968, 0.7886,
        0.5430, 0.6589, 0.6894, 0.5239, 0.7751, 0.5660, 0.5832, 0.6075, 0.4763,
        0.7108, 0.7125, 0.5238, 0.6683, 0.5538, 0.6191, 0.7145, 0.6611, 0.6788,
        0.7459, 0.7418, 0.6455, 0.6492, 0.6884, 0.7952, 0.7218, 0.7336, 0.6687,
        0.5981, 0.8177, 0.7637, 0.6822, 0.7940, 0.5627, 0.5844, 0.7129, 0.7136,
        0.5808, 0.6028, 0.6168, 0.7395, 0.8016, 0.7871, 0.4897, 0.6963, 0.7608,
        0.7063, 0.5874, 0.5158, 0.5249, 0.7707, 0.7789, 0.7543, 0.7334, 0.8036,
        0.7004, 0.6161, 0.7557, 0.5182, 0.7523, 0.7054, 0.6836, 0.5873, 0.6930,
        0.6415, 0.5843, 0.7300, 0.7037, 0.6505, 0.5907, 0.7230, 0.7401, 0.5917,
        0.7624, 0.5524, 0.6554, 0.6651, 0.5895, 0.6063, 0.5793, 0.4865, 0.7877,
        0.7443, 0.6010, 0.7536, 0.6875, 

In [8]:
import os
import shutil
import pickle as pk
from torch.utils.tensorboard import SummaryWriter

# 构建模型
model = TextRNN(parameter).to(parameter['cuda'])

# 确定训练模式
model.train()

# 确定优化器和损失
optimizer = torch.optim.SGD(model.parameters(),lr=0.1, momentum=0.95, nesterov=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.9)

# 准备迭代器
train_yield = batch_yield(parameter)

# 开始训练
loss_cal = []
min_loss = float('inf')
while 1:
        q,a,n,epoch,keys = next(train_yield)
        if keys:
            break
        q_emd,a_emd,n_emd = model(q,a,n)
        loss = nn.functional.triplet_margin_loss(q_emd, a_emd, n_emd,reduction='mean')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_cal.append(loss.item())
        if epoch is not None:
            if (epoch+1)%1 == 0:
                loss_cal = sum(loss_cal)/len(loss_cal)
                if loss_cal < min_loss:
                    min_loss = loss_cal
                    torch.save(model.state_dict(), 'grade.h5')
                print('epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, \
                                                       parameter['epoch'],loss_cal))
                optimizer.step()
            loss_cal = [loss.item()]



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1390/1390 [00:44<00:00, 30.94it/s]


epoch [1/100], Loss: 0.9616



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1390/1390 [01:02<00:00, 22.37it/s]


epoch [2/100], Loss: 0.9158



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1390/1390 [00:52<00:00, 26.23it/s]


...

epoch [98/100], Loss: 0.0289


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1390/1390 [00:32<00:00, 42.59it/s]


epoch [99/100], Loss: 0.0271


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1390/1390 [00:32<00:00, 42.58it/s]


epoch [100/100], Loss: 0.0332
